In [1]:
import numpy as np
import awkward as ak
import uproot
import vector
vector.register_awkward()

In [2]:
import os
import shutil
import zipfile
import tarfile
import urllib
import requests
from tqdm import tqdm

In [3]:
def _download(url, fname, chunk_size=1024):
    '''https://gist.github.com/yanqd0/c13ed29e29432e3cf3e7c38467f42f51'''
    resp = requests.get(url, stream=True)
    total = int(resp.headers.get('content-length', 0))
    with open(fname, 'wb') as file, tqdm(
        desc=fname,
        total=total,
        unit='iB',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for data in resp.iter_content(chunk_size=chunk_size):
            size = file.write(data)
            bar.update(size)

In [4]:
# Download the example file
example_file = 'JetClass_example_100k.root'
if not os.path.exists(example_file):
    _download('https://hqu.web.cern.ch/datasets/JetClass/example/JetClass_example_100k.root', example_file)

# Exploring the file

In [5]:
# Load the content from the file
tree = uproot.open(example_file)['tree']

In [6]:
# Display the content of the "tree"
tree.show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
part_px              | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
part_py              | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
part_pz              | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
part_energy          | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
part_deta            | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
part_dphi            | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
part_d0val           | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
part_d0err           | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
part_dzval           | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
part_dzerr           | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
part_charge          | std::

In [7]:
# Load all arrays in the tree
# Each array is a column of the table
table = tree.arrays()

In [8]:
# Arrays of a scalar type (bool/int/float) can be converted to a numpy array directly, e.g.
table['label_QCD'].to_numpy()

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [9]:
# Arrays of a vector type are loaded as a JaggedArray that has varying elements per row
table['part_px']

# A JaggedArray can be (zero-) padded to become a regular numpy array (see later)

<Array [[-125, -91.1, ... -0.735, -0.694]] type='100000 * var * float32'>

In [10]:
# Construct a Lorentz 4-vector from the (px, py, pz, energy) arrays
p4 = vector.zip({'px': table['part_px'], 'py': table['part_py'], 'pz': table['part_pz'], 'energy': table['part_energy']})

In [11]:
# Get the transverse momentum (pt)
p4.pt

<Array [[140, 95.3, 87.8, ... 1.3, 0.919]] type='100000 * var * float32'>

In [12]:
# Get the pseudorapidity (eta)
p4.eta

<Array [[-0.254, -0.403, ... -0.857, -0.935]] type='100000 * var * float32'>

In [13]:
# Get the azimuth angle (phi)
p4.phi

<Array [[2.67, 2.84, 2.81, ... -2.17, -2.43]] type='100000 * var * float32'>

In [14]:
def _pad(a, maxlen, value=0, dtype='float32'):
    if isinstance(a, np.ndarray) and a.ndim >= 2 and a.shape[1] == maxlen:
        return a
    elif isinstance(a, ak.Array):
        if a.ndim == 1:
            a = ak.unflatten(a, 1)
        a = ak.fill_none(ak.pad_none(a, maxlen, clip=True), value)
        return ak.values_astype(a, dtype)
    else:
        x = (np.ones((len(a), maxlen)) * value).astype(dtype)
        for idx, s in enumerate(a):
            if not len(s):
                continue
            trunc = s[:maxlen].astype(dtype)
            x[idx, :len(trunc)] = trunc
        return x


In [15]:
# Apply zero-padding and convert to a numpy array
_pad(p4.pt, maxlen=128).to_numpy()

array([[140.19296 ,  95.284584,  87.84807 , ...,   0.      ,   0.      ,
          0.      ],
       [244.67009 ,  62.332603,  45.159416, ...,   0.      ,   0.      ,
          0.      ],
       [143.15791 ,  91.48589 ,  25.372644, ...,   0.      ,   0.      ,
          0.      ],
       ...,
       [157.69547 , 101.245445,  79.816284, ...,   0.      ,   0.      ,
          0.      ],
       [ 88.65814 ,  80.69194 ,  79.14036 , ...,   0.      ,   0.      ,
          0.      ],
       [171.13641 , 121.71926 ,  59.68036 , ...,   0.      ,   0.      ,
          0.      ]], dtype=float32)

# Constructing features and labels

As you see previously with `tree.show()`, there are four groups of arrays with different prefixes:
 - `part_*`: JaggedArrays with features for each particle in a jet. These (and features constrcuted from them) are what we use for training in the Particle Transformer paper.
 - `label_*`: 1D numpy arrays one-hot truth labels for each jet. These are the target of the training.
 - *[Not used in the Particle Transformer paper]* `jet_*`: 1D numpy array with (high-level) features for each jet. These can also be used in the training, but since they are constructed from the particle-level features, it is not expected that they bring additional performance improvement.
 - *[Not used in the Particle Transformer paper]* `aux_*`: auxiliary truth information about the simulated particles for additional studies / interpretations. **SHOULD NOT be used in the training of any classifier.**

The code below illustrates how the input features and labels are constructed in the Particle Transformer paper.

(See also the yaml configuration: https://github.com/jet-universe/particle_transformer/blob/main/data/JetClass/JetClass_full.yaml)

In [16]:
def _clip(a, a_min, a_max):
    try:
        return np.clip(a, a_min, a_max)
    except ValueError:
        return ak.unflatten(np.clip(ak.flatten(a), a_min, a_max), ak.num(a))

In [17]:
def build_features_and_labels(tree, transform_features=True):
    
    # load arrays from the tree
    a = tree.arrays(filter_name=['part_*', 'jet_pt', 'jet_energy', 'label_*'])

    # compute new features
    a['part_mask'] = ak.ones_like(a['part_energy'])
    a['part_pt'] = np.hypot(a['part_px'], a['part_py'])
    a['part_pt_log'] = np.log(a['part_pt'])
    a['part_e_log'] = np.log(a['part_energy'])
    a['part_logptrel'] = np.log(a['part_pt']/a['jet_pt'])
    a['part_logerel'] = np.log(a['part_energy']/a['jet_energy'])
    a['part_deltaR'] = np.hypot(a['part_deta'], a['part_dphi'])
    a['part_d0'] = np.tanh(a['part_d0val'])
    a['part_dz'] = np.tanh(a['part_dzval'])

    # apply standardization
    if transform_features:
        a['part_pt_log'] = (a['part_pt_log'] - 1.7) * 0.7
        a['part_e_log'] = (a['part_e_log'] - 2.0) * 0.7
        a['part_logptrel'] = (a['part_logptrel'] - (-4.7)) * 0.7
        a['part_logerel'] = (a['part_logerel'] - (-4.7)) * 0.7
        a['part_deltaR'] = (a['part_deltaR'] - 0.2) * 4.0
        a['part_d0err'] = _clip(a['part_d0err'], 0, 1)
        a['part_dzerr'] = _clip(a['part_dzerr'], 0, 1)

    feature_list = {
        'pf_points': ['part_deta', 'part_dphi'], # not used in ParT
        'pf_features': [
            'part_pt_log', 
            'part_e_log',
            'part_logptrel',
            'part_logerel',
            'part_deltaR',
            'part_charge',
            'part_isChargedHadron',
            'part_isNeutralHadron',
            'part_isPhoton',
            'part_isElectron',
            'part_isMuon',
            'part_d0',
            'part_d0err',
            'part_dz',
            'part_dzerr',
            'part_deta',
            'part_dphi',
        ],
        'pf_vectors': [
            'part_px',
            'part_py',
            'part_pz',
            'part_energy',
        ],
        'pf_mask': ['part_mask']
    }

    out = {}
    for k, names in feature_list.items():
        out[k] = np.stack([_pad(a[n], maxlen=128).to_numpy() for n in names], axis=1)

    label_list = ['label_QCD', 'label_Hbb', 'label_Hcc', 'label_Hgg', 'label_H4q', 'label_Hqql', 'label_Zqq', 'label_Wqq', 'label_Tbqq', 'label_Tbl']
    out['label'] = np.stack([a[n].to_numpy().astype('int') for n in label_list], axis=1)
    
    return out

In [18]:
build_features_and_labels(tree)

{'pf_points': array([[[-0.07242048,  0.07607916,  0.08601749, ...,  0.        ,
           0.        ,  0.        ],
         [-0.08581114,  0.09253383,  0.06340456, ...,  0.        ,
           0.        ,  0.        ]],
 
        [[ 0.01535046, -0.00294232,  0.03290105, ...,  0.        ,
           0.        ,  0.        ],
         [-0.04896092, -0.04723394, -0.06385756, ...,  0.        ,
           0.        ,  0.        ]],
 
        [[-0.13630104, -0.14928365, -0.17035806, ...,  0.        ,
           0.        ,  0.        ],
         [-0.01668766, -0.02666983, -0.01680285, ...,  0.        ,
           0.        ,  0.        ]],
 
        ...,
 
        [[ 0.07362503,  0.09081   , -0.15697095, ...,  0.        ,
           0.        ,  0.        ],
         [ 0.01177871,  0.02063447, -0.01410705, ...,  0.        ,
           0.        ,  0.        ]],
 
        [[ 0.03936064,  0.04921722,  0.03772318, ...,  0.        ,
           0.        ,  0.        ],
         [ 0.04601908,  